In [1]:
# MLP 

In [22]:
from keras.datasets import mnist  # 导入mnist数据库， mnist是常用的手写数字库
from keras.models import Sequential # 导入序列模型
from keras.layers.core import Dense,Dropout,Activation # 导入全连接层Dense， 激活层Activation, dropout层
from keras.optimizers import Adam,SGD # 导入优化器Adam
from keras.utils import np_utils # # 导入numpy工具，主要是用to_categorical来转换类别向量

In [23]:
import keras

In [24]:
keras.__version__

'2.2.4'

In [25]:
batch_size = 128 #设置batch的大小
nb_classes = 10 #设置类别的个数
nb_epoch = 15 #设置epoch的个数

In [26]:
# keras中的mnist数据集已经被划分成了60,000个训练集，10,000个测试集的形式，按以下格式调用即可
(X_train, y_train), (X_test, y_test) = mnist.load_data()

In [27]:
X_train.shape

(60000, 28, 28)

In [28]:
X_test.shape

(10000, 28, 28)

In [29]:
# X_train原本是一个60000*28*28的三维向量，将其转换为60000*784的二维向量
X_train = X_train.reshape(-1, 784) 
X_test = X_test.reshape(-1, 784)
X_train = X_train.astype('float32') # 将X_train, X_test的数据格式转为float32存储
X_test = X_test.astype('float32')
#归一化
X_train /= 255
X_test /= 255

In [32]:
X_train[0]

array([0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.     

In [33]:
y_train

array([5, 0, 4, ..., 5, 6, 8], dtype=uint8)

In [35]:
# one-hot编码
Y_train = np_utils.to_categorical(y_train, nb_classes)
Y_test = np_utils.to_categorical(y_test, nb_classes)

In [39]:
Y_train[0]

array([0., 0., 0., 0., 0., 1., 0., 0., 0., 0.], dtype=float32)

In [13]:
# '''
# 模型需要知道输入数据的shape，
# 因此，Sequential的第一层需要接受一个关于输入数据shape的参数，
# 后面的各个层则可以自动推导出中间数据的shape，
# 因此不需要为每个层都指定这个参数
# ''' 

In [46]:
# 输入层有784个神经元
# 第一个隐层有128个神经元，激活函数为ReLu，Dropout比例为0.2
model = Sequential()
model.add(Dense(128, input_shape=(784,)))
model.add(Activation('relu'))
model.add(Dropout(0.2))

# 第二个隐层有128个神经元，激活函数为ReLu，Dropout比例为0.2

model.add(Dense(128))
model.add(Activation('relu'))
model.add(Dropout(0.2))

# 输出层有10个神经元，激活函数为SoftMax，得到分类结果

model.add(Dense(10))
model.add(Activation('softmax'))

In [47]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_7 (Dense)              (None, 128)               100480    
_________________________________________________________________
activation_7 (Activation)    (None, 128)               0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_8 (Dense)              (None, 128)               16512     
_________________________________________________________________
activation_8 (Activation)    (None, 128)               0         
_________________________________________________________________
dropout_2 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_9 (Dense)              (None, 10)                1290      
__________

In [48]:
784*128+128


100480

In [49]:
#总共参数数量为784*h1_level+h1_level + h2_level*h2_level+h2_level + h2_level*10+10 
h_level = 128 
784*h_level+h_level 

100480

In [50]:
h_level*h_level+h_level

16512

In [51]:
h_level*10 +10

1290

In [52]:
784*h_level+h_level  +  h_level*h_level+h_level  + h_level*10 +10

118282

In [53]:

# '''
# 配置模型的学习过程
# compile接收三个参数：
# 1.优化器optimizer：参数可指定为已预定义的优化器名，如rmsprop、adagrad，adam
# 或一个Optimizer类对象，如此处的RMSprop()
# 2.损失函数loss：参数为模型试图最小化的目标函数，可为预定义的损失函数，
# 如categorical_crossentropy、mse，也可以为一个损失函数
# 3.指标列表：对于分类问题，一般将该列表设置为metrics=['accuracy']
# '''
model.compile(loss='categorical_crossentropy',optimizer=Adam(), metrics=['accuracy'])

In [54]:
# '''
# 训练模型
# batch_size：指定梯度下降时每个batch包含的样本数
# nb_epoch：训练的轮数，nb指number of
# verbose：日志显示，0为不在标准输出流输出日志信息，1为输出进度条记录，2为epoch输出一行记录
# validation_data：指定验证集
# fit函数返回一个History的对象，其History.history属性记录了损失函数和其他指标的数值随epoch变化的情况，
# 如果有验证集的话，也包含了验证集的这些指标变化情况
# '''

history = model.fit(
                    X_train
                    , Y_train
                    ,batch_size=batch_size
                    , verbose=1
                    ,nb_epoch=30
                    , validation_data=(X_test, Y_test)
                   )

/Library/Frameworks/Python.framework/Versions/3.4/lib/python3.4/site-packages/ipykernel/__main__.py:17: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.


Train on 60000 samples, validate on 10000 samples
Epoch 1/30
60000/60000 [==============================] - 4s 62us/step - loss: 0.4067 - acc: 0.8795 - val_loss: 0.1645 - val_acc: 0.9502
Epoch 2/30
60000/60000 [==============================] - 3s 49us/step - loss: 0.1772 - acc: 0.9465 - val_loss: 0.1166 - val_acc: 0.9643
Epoch 3/30
60000/60000 [==============================] - 3s 50us/step - loss: 0.1358 - acc: 0.9601 - val_loss: 0.1011 - val_acc: 0.9695
Epoch 4/30
60000/60000 [==============================] - 3s 49us/step - loss: 0.1110 - acc: 0.9657 - val_loss: 0.0825 - val_acc: 0.9746
Epoch 5/30
60000/60000 [==============================] - 3s 50us/step - loss: 0.0939 - acc: 0.9707 - val_loss: 0.0825 - val_acc: 0.9741
Epoch 6/30
60000/60000 [==============================] - 3s 50us/step - loss: 0.0842 - acc: 0.9739 - val_loss: 0.0807 - val_acc: 0.9753
Epoch 7/30
60000/60000 [==============================] - 3s 49us/step - loss: 0.0760 - acc: 0.9756 - val_loss: 0.0767 - val_acc